# 0. Important libraries used

In [ ]:
# array creation and manipulation
import numpy as np

# sampling
import random

# plotting
import matplotlib.pyplot as plt

# counting
import itertools

# math
import math as mt

# directory creation
import os

# file manipulation
import glob

# image compilation
import cv2

# data storing
import pandas as pd

# data interpolation
from scipy.interpolate import interp1d

# plotting colors
from matplotlib import colors
import matplotlib

# I. Cell information functions

This group consists of functions collecting certain information regarding the 
CA grid, namely:

   > `totalCells`: number of cells of the whole CA grid.
   
   > `ringCellCounter`: number of cells in the outermost ring of the CA grid.
   
   > `locateCell`: the location of the cell in the grid in terms of which ring it
                   is located at what cell number within the ring it is situated.
                    
## a. Total cell count (whole grid)

In [ ]:
def totalCells(ringCount, centerCellCount):
    """
    Returns the total number of cells of a grid with stated 
    number of rings and number of cells in the innermost ring.
    
        Parameters
        ----------
            ringCount : int
                The number of rings for our grid.
            centerCellCount : int
                The number of cells for the innermost ring.
        
        Returns
        -------
            total : int
                The total number of cells in the grid.   
    """
    total = 0
    for i in range(ringCount):
        total += centerCellCount + 2 * centerCellCount * i
        
    return total

## b. Total cell count (ring in the grid)

In [ ]:
def ringCellCounter(ringCount, centerCellCount):
    """
    Returns the total number of cells in the outermost ring.
    
        Parameters
        ----------
            ringCount : int
                The number of rings for our grid.
            centerCellCount : int
                The number of cells for the innermost ring.
        
        Returns
        -------
            ringCellCount : int
                The number of cells in any ring.
    """
    ringCellCount = centerCellCount + 2 * centerCellCount * (ringCount - 1)
    
    return ringCellCount

## c. Cell profile

In [ ]:
def locateCell(ringNumber, cellNumber):
    """
    Returns information regarding a target cell given its position
    in terms of ringNumber (which ring it is in) and cellNumber 
    (the number designation of a target cell in a given ring).
    
        Parameters
        ----------
            ringNumber : int
                The ring where a target cell is located.
            cellNumber : int
                The number designation of a target cell
                in a ring.
        
        Returns
        -------
            cellCount : int
                The number of cells in a specified ring.
            cellSectorSize : int
                How wide a cell is theta-wise.
            cellLowSector : float
                The angular position of the low-valued sector
                of the target cell.
            cellUpSector : float
                The angular position of the high-valued sector
                of the target cell.
    """
    # number of cells in the specified ring
    cellCount = centerCellCount + 2 * centerCellCount * (ringNumber)
    
    # assertion to make inputs valid
    assert ringNumber >= 0, "Ring number invalid."
    assert ringNumber <= ringCount - 1, "Ring number invalid."
    
    # position should be between 0 and maximum allowed 
    # cells in a ring
    while cellNumber < 0:
        cellNumber += cellCount
    while cellNumber > cellCount - 1:
        cellNumber -= cellCount
    
    # size of sector theta-wise
    cellSectorSize = 360/cellCount
    
    # sector dimensions of the target cell
    cellLowSector = cellSectorSize * cellNumber
    cellUpSector = cellSectorSize * (cellNumber + 1)
    
    return cellCount, cellSectorSize, cellLowSector, cellUpSector

# II. Initialization functions

This group consists of functions that initialize the CA grid

   > `zeroGrid`: creates the CA grid and initializes it with zero values and
                 garbage values. 0 indicates inactive cells and garbage values 
                 indicate cells that are not part of the actual CA grid.
   
   > `initialState`: populates the cell with ones values. 1 indicates active
                     cells.

## a. Zero grid

In [ ]:
def zeroGrid(ringCount, centerCellCount):
    """
    Returns an array of zeros to where we will store grid
    data. -1 values in the array are just garbage values.
    
        Parameters
        ----------
            ringCount : int
                The number of rings for our grid
            centerCellCount : int
                The number of cells for the innermost ring
        
        Returns
        -------
            zeroGrid : int (array)
                An array of zeros to where we will store
                cell information. 
    """
    maxCells = ringCellCounter(ringCount, centerCellCount)
    
    # first ring for polar grid
    zeroGrid = np.zeros(maxCells, dtype=int)
    # garbage data
    zeroGrid[centerCellCount:] = -1
    
    # adding more rows that represent data for outer ring cells
    for i in range(1, ringCount):
        newRing = np.zeros(maxCells, dtype=int)
        newRing[centerCellCount + 2 * centerCellCount * i:] = -1
        zeroGrid = np.vstack([zeroGrid, newRing])
    
    return zeroGrid

## b. Initial grid

In [ ]:
def initialState(popAlive, zeroGrid):
    """
    Returns an array of zeros and garbage values populated by 
    1s.
    
        Parameters
        ----------
            popAlive : float
                The percentage of cells in the grid that will
                be populated by 1s.
            zeroGrid : int (array)
                The initial grid containing 0s.
        
        Returns
        -------
            zeroGrid : int (array)
                An array of zeros that have been populated by
                1s serving as the initial state of the model. 
    """

    totCells = totalCells(ringCount, centerCellCount)
    numberPopulated = int(totCells * popAlive / 100)
    
    # obtaining the cell numbers that will be initially populated
    cellsPopulated = random.sample(range(totalCells(noNucleusRatio, 3), totCells), numberPopulated)
    cellsPopulated.sort()
    
    # population mechanism (separate function can be made for this)
    for j in cellsPopulated:
        for i in range(ringCount):
            maxCells = ringCellCounter(i + 1, centerCellCount)
            if j <= maxCells - 1:
                zeroGrid[i, j] = 1
                break
            else:
                j -= maxCells
        
    return zeroGrid

# III. Neighbor detection function

This function is responsible for the neighbor detection algorithm within the CA grid.

In [ ]:
def checkNeighbors(ringNumber, cellNumber):
    """
    Returns an array of neighbors given the information about
    the target cell.
    
        Parameters
        ----------
            ringNumber : int
                The ring where a target cell is located.
            cellNumber : int
                The number designation of a target cell
                in a ring.
        
        Returns
        -------
            uniqueNeighbors : int (array)
                An array that holds the locations of the
                neighbors of the target cell.
    """
    # number of cells in the specified ring
    cellCount = locateCell(ringNumber, cellNumber)[0]
    
    # assertion to make inputs valid
    assert ringNumber >= 0, "Ring number invalid."
    assert ringNumber <= ringCount - 1, "Ring number invalid."
    
    # position should be between 0 and maximum allowed 
    # cells in a ring
    while cellNumber < 0:
        cellNumber += cellCount
    while cellNumber > cellCount - 1:
        cellNumber -= cellCount
    
    # array that contains the neighbor coordinates
    neighbors = []
    
    # current ring neighbors
    leftNeighbor = [ringNumber, cellNumber - 1]
    rightNeighbor = [ringNumber, cellNumber + 1]
    
    if leftNeighbor[1] < 0:
        leftNeighbor[1] = cellCount - 1
    if rightNeighbor[1] > cellCount - 1:
        rightNeighbor[1] = 0

    neighbors.append(leftNeighbor)
    neighbors.append(rightNeighbor)
    
    # side positions of the target cell
    cellLowSector = locateCell(ringNumber, cellNumber)[2]
    cellUpSector = locateCell(ringNumber, cellNumber)[3]
    
    # inner ring neighbors
    if ringNumber > 0:
        #offsetInner = 0
        offsetInner = rotationIncrements[ringNumber] - rotationIncrements[ringNumber - 1] 
        cellCountInner = locateCell(ringNumber - 1, cellNumber)[0]
        cellSizeInner = locateCell(ringNumber - 1, cellNumber)[1]
        
        lowSectorDetectionInner = mt.floor((cellLowSector + offsetInner)/cellSizeInner)
        upSectorDetectionInner = mt.floor((cellUpSector + offsetInner)/cellSizeInner)
        
        leftInnerNeighbor = [ringNumber - 1, lowSectorDetectionInner]
        rightInnerNeighbor = [ringNumber - 1, upSectorDetectionInner]
        
        if cellUpSector/cellSizeInner == float(upSectorDetectionInner):
            rightInnerNeighbor[1] -= 1
            
        while leftInnerNeighbor[1] < 0:
            leftInnerNeighbor[1] += cellCountInner
        while leftInnerNeighbor[1] >= cellCountInner:
            leftInnerNeighbor[1] -= cellCountInner
        while rightInnerNeighbor[1] < 0:
            rightInnerNeighbor[1] += cellCountInner
        while rightInnerNeighbor[1] >= cellCountInner:
            rightInnerNeighbor[1] -= cellCountInner
            
        neighbors.append(leftInnerNeighbor)
        neighbors.append(rightInnerNeighbor)

    # outer ring neighbors
    if ringNumber < ringCount - 1:
        #offsetOuter = 0
        offsetOuter = rotationIncrements[ringNumber] - rotationIncrements[ringNumber + 1]
        cellCountOuter = locateCell(ringNumber + 1, cellNumber)[0]
        cellSizeOuter = locateCell(ringNumber + 1, cellNumber)[1]
        
        lowSectorDetectionOuter = mt.floor((cellLowSector + offsetOuter)/cellSizeOuter)
        upSectorDetectionOuter = mt.floor((cellUpSector + offsetOuter)/cellSizeOuter)
        
        if cellUpSector/cellSizeOuter == float(upSectorDetectionOuter):
            upSectorDetectionOuter -= 1
        
        for neighbor in range(lowSectorDetectionOuter, 
                              upSectorDetectionOuter + 1,
                              1):
            while neighbor < 0:
                neighbor += cellCountOuter
            while neighbor >= cellCountOuter:
                neighbor -= cellCountOuter
                
            neighbors.append([ringNumber + 1, neighbor])
    
    # removing duplicate detections
    uniqueNeighbors = []
    
    for i in neighbors:
        if not i in uniqueNeighbors:
            uniqueNeighbors.append(i)
            
    return uniqueNeighbors

# IV. Star formation model and chain counting algorithm

This function is contains the SSPSF model algorithm based on Gerola & Seiden's work.

In [ ]:
def chainFormation(timesteps, initial):
    
    """
    Returns an array of neighbors given the information about
    the target cell.
    
        Parameters
        ----------
            timesteps : int
                The number of time steps for the simulation.
            initial : int (array)
                The number designation of a target cell in 
                a ring.
        
        Returns
        -------
            evolution_activity : int (array)
                An array that holds the evolution of the 
                initial state over the number of time
                steps.
            chainInfoFinal : int (array)
                An array that holds the chain profiles of
                all generated chains.
            rotationAccumulated : float (array)
                An array that holds the amount of rotation
                gained by every ring of cells at every time
                step.
            triggeredCells : int (array)
                An array that holds all triggered cells in
                the simulation (counting multiplicities).
            triggeredDensity : int (array)
                An array that holds the number of cells that
                got triggered in every time step.
    """
    
    # rotation array for plotting
    rotationAccumulated = np.zeros(ringCount)
    
    # array that will contain the iteration cell data for all timesteps
    evolution_activity = np.copy(initial)
    
    # array that contains cumulative chain growth and cell death within chains
    cumul_activity = np.copy(initial)
    
    # array to store all chains, including unfinished chains
    chainInfo = []
    
    # array that stores how all cells that were triggered including repeated triggers
    triggeredCells = []
    
    # array that stores the amount of cells that get triggered every timestep
    triggeredDensity = []
    
    # initial dictionary of all source cells that will trigger neighbors
    sourceCellsDict = {}
    
    # locations of initially active cells and making a chain profile for each
    cellLocation = np.where(cumul_activity == 1)
    for cell in range(len(cellLocation[0])):
        # identifying each triggered cell
        ringNum = cellLocation[0][cell]
        cellNum = cellLocation[1][cell]

        # initializing list to where we will store different information about a chain
        currentGenCells = []
        chainSize = []
        chainDuration = []
        chainGrowth = []
        chainStart = []

        # starting / source cell
        currentGenCells.append([ringNum,cellNum])
        chainSize.append([ringNum,cellNum])

        # collecting the duration of chain in terms of timesteps
        chainDuration.append(1)

        # collecting the amount of cells that get triggered by a chain every timestep
        chainGrowth.append(1)

        # inserting the time when the chain started in the simulation
        chainStart.append(0)

        # appending everything into one superlist
        chainInfo.append([currentGenCells, chainSize, chainDuration, chainGrowth, chainStart])

        # adding a dictionary key to where we will store the triggered neighbors as dictionary values
        sourceCellsDict[str(ringNum) + '.' + str(cellNum)] = []

        # appending to the array of triggered cells
        triggeredCells.append([ringNum, cellNum])
    
    # array to store terminating chains
    chainInfoFinal = []
    
    for i in range(timesteps):
        
        # differential rotation update per timestep
        for ring in range(ringCount):
            rotationIncrements[ring] += rotationCurves[ring]
            if rotationIncrements[ring] > 360:
                rotationIncrements[ring] -= 360
            #if rotationIncrements[ring] == 0:
                #rotationAccumulator[ring] = 360
            rotationAccumulatedCurrent[ring] += rotationCurves[ring]
               
        # array that will hold all changes in the cell status incorporated later on
        changesToCumul = []

        # checking every source cell of all chains
        for sourceCell in sourceCellsDict.keys():
            
            # converting the dictionary key into cell coordinates
            split_num = str(sourceCell).split('.')
            ringNum = int(split_num[0])
            cellNum = int(split_num[1])
            
            # looping over all neighbors of source cell
            for neighbor in checkNeighbors(ringNum, cellNum):
                
                # a factor multiplied to a constant stimulated probability
                if refTime == 0:
                    probFactor = 1
                else:
                    if (cumul_activity[neighbor[0], neighbor[1]] == 0) or (cumul_activity[neighbor[0], neighbor[1]] > refTime):
                        probFactor = 1
                    else:
                        probFactor = (cumul_activity[neighbor[0], neighbor[1]] - 1) / refTime

                # considering gerola and seiden variant
                if neighbor[0] == noNucleusRatio - 1:
                    probFactor = 0
                
                # if probability of triggering succeeded:
                if random.random() <= stimProb * probFactor:
                    
                    # triggered neighbors appended to the array of triggered cells
                    triggeredCells.append([neighbor[0], neighbor[1]])
                    
                    # triggered neighbors appended to the corresponding dictionary values
                    sourceCellsDict[sourceCell].append(neighbor)
                    
                    # triggered neighbors appended to cell status updates array
                    if [neighbor[0], neighbor[1]] not in changesToCumul:
                        changesToCumul.append([neighbor[0], neighbor[1]])
                             
        # array that will hold all new source cells for the next timestep
        newSourceCells = []
        
        # looping over every chain profile
        for subArray in chainInfo:
            
            # checking every array of source cells in a chain
            newCells = []
            for sourceCell in subArray[0][0:len(subArray[0])]:
                sourceCellStr = str(sourceCell[0]) + '.' + str(sourceCell[1])
                for connectedCells in sourceCellsDict[sourceCellStr]:
                    if connectedCells not in newCells:
                        newCells.append(connectedCells)
                        
                    # adding all unique cells that grow in all chains
                    if connectedCells not in newSourceCells:
                        newSourceCells.append(connectedCells)
                        
                # removing the source cells from left to right to accommodate for the source cells of the next generation
                subArray[0].pop(0)
            
            # storing cells in this subArray dedicated for chain measurements
            for newCell in newCells:
                subArray[0].append(newCell)   
                subArray[1].append(newCell)
            
            if len(subArray[0]) > 0:
                subArray[2][0] += 1
                subArray[3].append(len(newCells))
                
            if len(subArray[0]) == 0:
                chainInfoFinal.append([subArray[1], subArray[2], subArray[3], subArray[4]])
                subArray[1] = []
                subArray[2] = []
                subArray[3] = []
                subArray[4] = []
        try:
            while True:
                chainInfo.remove([[], [], [], [], []])
        except ValueError:
            pass

        
        # appending the number of triggered cells per timestep
        triggeredDensity.append(len(changesToCumul))
        
        # deduct a timestep in the cumul array to indicate lifetime / aging
        for j in range(ringCount):
            maxCells = ringCellCounter(j + 1, centerCellCount)
            for k in range(maxCells):
                if cumul_activity[j, k] > 0:
                    cumul_activity[j, k] += 1
                    
        # applying changes to cumul array
        for cells in changesToCumul:
            cumul_activity[cells[0], cells[1]] = 1
            
        # stacking every iteration into one array
        evolution_activity = np.vstack((evolution_activity, cumul_activity))
        
        # rotation updates for plotting
        rotationAccumulated = np.vstack((rotationAccumulated, rotationAccumulatedCurrent))

        # all inactive cells will be potential cells to undergo spontaneous star formation
        spontCandidates = []
        
        # identifying all inactive cells
        cellLocSpont = np.where((cumul_activity == 0) | (cumul_activity > refTime))
        
        # placing their coordinates in the candidates array
        for cell in range(len(cellLocSpont[0])):
            if cellLocSpont[0][cell] > noNucleusRatio - 1:
                ringNumSpont = cellLocSpont[0][cell]
                cellNumSpont = cellLocSpont[1][cell]
                spontCandidates.append([ringNumSpont,cellNumSpont])
            
        # identifying winning candidates    
        if spontSize <= len(spontCandidates):
            spontWinners = random.sample(spontCandidates, spontSize)
            spontWinners.sort(key=lambda tup: tup[0])
            
            # making a new chain profile for every winning candidate
            for winner in spontWinners:
                newSource = [[],[],[],[],[]]
                newSource[0].append(winner)
                newSource[1].append(winner)
                newSource[2].append(1)
                newSource[3].append(1)
                newSource[4].append(i + 1)
                chainInfo.append(newSource)
                
                # updating their cell status
                cumul_activity[winner[0], winner[1]] = 1
                
                # adding winner to list of new source cells
                newSourceCells.append(winner)
        
        # emptying the source cell dictionary to accommodate for the next generation of source cells
        sourceCellsDict = {}
        
        # adding all next generation source cells in the dictionary
        for newSourceCell in newSourceCells:
            ringNumNew = newSourceCell[0]
            cellNumNew = newSourceCell[1]
            newSourceCellStr = str(ringNumNew) + '.' + str(cellNumNew)
            sourceCellsDict[newSourceCellStr] = []
        
        if i == timesteps - 1:
            for subArray in chainInfo:
                chainInfoFinal.append([subArray[1],subArray[2], subArray[3], subArray[4]])
            
    # splitting for easier plotting later on
    evolution_activity = np.vsplit(evolution_activity, timesteps + 1)
  
    return evolution_activity, chainInfoFinal, rotationAccumulated, triggeredCells, triggeredDensity

# V. Visualization

This group consists of functions that visualize the evolution of the model.

   > `plotAndSave`: plots the individiual arrays of information and compiling
                    those plots in a folder.
   
   > `animateGOL`: creates a video animation of the compiled plots.

In [ ]:
def plotAndSave(cellData, folderName):
    """
    Saves plots of each iteration into image files into 
    a specified folder.
    
        Parameters
        ----------
            cellData : int (array(array))
                The array holding the evolution information.
            folderName : str
                the name of the folder to where the plot
                image files will be saved.
    """
    
    # plotting the current iteration
    for idx, state in enumerate(cellData):
        fig = plt.figure(idx + 1, figsize=(20, 20))
        plt.xticks([])
        plt.yticks([])
        ax_cart = fig.add_axes([0.1, 0.1, 0.8, 0.8])
        ax_cart.set_xlim(-maxRad, maxRad)
        ax_cart.set_ylim(-maxRad, maxRad)
        ax_cart.set_xlabel('radius [kpc]', fontsize=30)
        ax_cart.set_ylabel('radius [kpc]', fontsize=30)
        ax_cart.tick_params(axis='both', which='major', labelsize=20)
        ax = fig.add_axes([0.08, 0.08, 0.84, 0.84], polar=True, frameon=False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False)
        
        # colors depicting inactive and active regions
        # black: inactive; navy: active
        states = ['black', 'navy']
        
        # for each ring in the disk...
        for i in range(state.shape[0]):
            maxCells = ringCellCounter(i + 1, centerCellCount)
            
            # for each cell in the ring...
            for j in range(maxCells):
                if state[i,j] == refTime:
                    ax.bar((j) * 2 * mt.pi / maxCells + mt.pi/2 + (rotations[idx, i])*mt.pi/180,
                           bottom = i,
                           height = 1, 
                           width = 2 * mt.pi / maxCells,
                           align = 'edge',
                           color = 'red')
               
        fig.suptitle('Cell status at time: '+str(idx*15)+' million years', fontsize=30, y = .95)
        folderLoc = mainFolderName + folderName + '/'
        filename = str(idx + 1) + '.jpg'
        plt.savefig(folderLoc + filename)
        fig.clf()
        plt.close("all")

In [ ]:
def animateGOL(folderName, videoName):
    """
    Takes images in a specified folder and compiles them
    in a video format.
    
        Parameters
        ----------
            folderName : str
                The name of the folder holding the image
                files.
            videoName : str
                The desired name for the video file 
                produced.
    """
    fileCount = len(glob.glob(mainFolderName + folderName + '/*.jpg'))
    img_array = []
    for fileNumber in range(fileCount):
        filename = mainFolderName + folderName + '/' + str(fileNumber + 1) + '.jpg'
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    
    # video creation
    out = cv2.VideoWriter(mainFolderName + videoName + '.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
 
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

# VI. Other handy functions

This group consists of other functions that were used in the model.

   > `rotCurveConvert`: converts the distance in the rotation curve
                        data in terms of angular distance.
   
   > `createFolder`: creates a folder to store all runs with the
                     same parameters.
   
## a. Rotation curve conversion

In [ ]:
def rotCurveConvert(rotationCurve, deltaT):
    """
    Converts the given rotation curve data in terms of
    angular position increment values.
    
        Parameters
        ----------
            rotationCurve : float (array)
                An array that holds the rotation curve
                data to be converted.
            deltaT : float
                The timescale per timestep used for
                conversion.
        
        Returns
        -------
            distance : float (array)
                The converted rotation curve data.
    """
    distance = rotationCurve * deltaT
    for i in range(distance.size):
        distance[i] = distance[i] / (2 * mt.pi * radiiCentered[i] * 3.086e+16) * 360
    return distance

## b. Creating folder for a specific run

In [ ]:
def createFolder(mainFolderName):
    """
    Creates a directory where all runs of the same
    parameter combinations are stored.
    
        Parameters
        ----------
            mainFolderName : string
                The name of the folder.
    """
    os.mkdir(mainFolderName)